In [82]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [83]:
sns.set(color_codes=True)

In [84]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [85]:
other_df = pd.get_dummies(train['DetectedCamera']) 
other_df_test = pd.get_dummies(test['DetectedCamera'])

In [86]:
merged_df = pd.concat([other_df,train],1) 
merged_df_test = pd.concat([other_df_test,test],1)

In [88]:
le = LabelEncoder()
merged_df['SignFacing (Target)']=le.fit_transform(merged_df['SignFacing (Target)'])


In [89]:
train = merged_df 
test = merged_df_test

In [90]:
def func1(x):
    if x=='Left':
        return -90
    elif x=='Right':
        return -90
    else :
        return 0

train['NewAngleOfSign'] = train['AngleOfSign']- train['DetectedCamera'].apply(func1);
test['NewAngleOfSign'] = test['AngleOfSign']- test['DetectedCamera'].apply(func1);


In [91]:
train_label = train['SignFacing (Target)']

training = train.drop(labels=['Id', 'DetectedCamera', 'SignFacing (Target)', 'AngleOfSign'], axis=1)
testing = test.drop(labels=['Id', 'DetectedCamera', 'AngleOfSign'], axis=1)

In [93]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import xgboost as xgb


## Grid Search

In [ ]:
# kfold = KFold(n_splits=5, random_state=7)
# model = xgb.XGBClassifier(objective='multi:softprob', nthread=-1, reg_lambda=1, seed=2017, subsample=0.999, gamma=0.2, learning_rate=0.15, max_depth=4)
# params_grid = { 'n_estimators':list([70,75,80,85,90]), 'min_child_weight':[3,4,5], 'colsample_bytree':list([1,0.8,0.6]), 'colsample_bylevel':list([0.7,0.6,0.5])  }
# grid1 = GridSearchCV(estimator=model, param_grid=params_grid, n_jobs=-1, scoring='log_loss', cv=kfold)
# grid1.fit(training, train_label)

/home/clabuser/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/clabuser/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/clabuser/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/clabuser/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/home/clabuser/anaconda2/lib/python2.7/site-packages/sklearn/metrics/sco

In [ ]:
# print grid1.best_estimator_
# print grid1.best_params_
# print grid1.best_score_

In [ ]:
# new_model = grid1.best_estimator_
# new_model.fit(training, train_label)
# prediction = new_model.predict_proba(testing)

## Prep For XGBoost

In [94]:
xtrain, xval, ytrain, yval = train_test_split(training,train_label,test_size=0.4,random_state=7)

In [95]:
preds = testing.columns
DMtraining = xgb.DMatrix(training, train_label, feature_names=preds)
DMtrain = xgb.DMatrix(xtrain,ytrain,feature_names=preds)
DMval = xgb.DMatrix(xval,yval,feature_names=preds)
DMtest = xgb.DMatrix(testing, feature_names=preds)

In [96]:
xgb_params={
    'eta':0.15,
    'max_depth':4,
    'colsample_bytree':1.0,
    'colsample_bylevel':0.6,
    'subsample' : 0.9,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'min_child_weight':4,
    'silent':1,
    'seed':2017,
    'nthread':4,
    'gamma' : 0.2,
    'num_class':4,
    'lambda':7
}

In [97]:
watchlist = [(DMtrain, 'Train'), (DMval,'Validation')]
num_rounds= 2500
model = xgb.train(xgb_params, DMtrain, num_rounds, watchlist, early_stopping_rounds=80, verbose_eval=5)

[0]	Train-mlogloss:1.12759	Validation-mlogloss:1.12829
Multiple eval metrics have been passed: 'Validation-mlogloss' will be used for early stopping.

Will train until Validation-mlogloss hasn't improved in 80 rounds.
[5]	Train-mlogloss:0.517897	Validation-mlogloss:0.521021
[10]	Train-mlogloss:0.290259	Validation-mlogloss:0.295645
[15]	Train-mlogloss:0.191587	Validation-mlogloss:0.198971
[20]	Train-mlogloss:0.143891	Validation-mlogloss:0.15358
[25]	Train-mlogloss:0.121231	Validation-mlogloss:0.132456
[30]	Train-mlogloss:0.110023	Validation-mlogloss:0.122741
[35]	Train-mlogloss:0.103871	Validation-mlogloss:0.117519
[40]	Train-mlogloss:0.100038	Validation-mlogloss:0.114266
[45]	Train-mlogloss:0.097484	Validation-mlogloss:0.112451
[50]	Train-mlogloss:0.095612	Validation-mlogloss:0.111344
[55]	Train-mlogloss:0.094165	Validation-mlogloss:0.110381
[60]	Train-mlogloss:0.092807	Validation-mlogloss:0.109818
[65]	Train-mlogloss:0.09158	Validation-mlogloss:0.109264
[70]	Train-mlogloss:0.090636	Va

In [98]:
model = xgb.train(xgb_params, DMtraining,num_boost_round=int(110/0.95))
prediction = model.predict(DMtest)

In [99]:
subm_file = pd.read_csv('sample_submission.csv')

In [100]:
subm_file['Id'] = test['Id']
subm_file['Front'] = prediction[:,0]
subm_file['Left'] = prediction[:,1]
subm_file['Rear'] = prediction[:,2]
subm_file['Right'] = prediction[:,3]

In [101]:
subm_file.to_csv('subm_new_ang2.csv', index=None)